# Mesh convergence \& method of manufactured solutions

boundary value problem on $\textbf{x}\in\Omega$

$$\nabla^2 u = f(\textbf{x})$$

manufactured solution

$$u_{\text{e}}(\textbf{x})\implies f=\nabla^2u_{\text{e}}~,~u_{\text{D}}=u_{\text{e}}~,~\partial\Omega_{\text{D}}=\partial\Omega$$